In [22]:
import numpy as np
from PIL import Image
import colorsys
from pathlib import Path
import glob
import ray
import time

In [23]:
# From pixelSort.ipynb
def get_image_pixel(imagePIL, resize="None"):
    if resize == "None":   
        return imagePIL, list(imagePIL.getdata())# np.array(imagePIL.getdata()) #list(imagePIL.getdata())
    else:
        org_size = imagePIL.size
        pixelate_lvl = resize
        imagePIL = imagePIL.resize(size=(org_size[0] // pixelate_lvl, org_size[1] // pixelate_lvl), resample=0)# imagePIL.resize((resize,resize), resample=Image.BILINEAR)
        return imagePIL, np.array(imagePIL.getdata()) # list(imagePIL.getdata())

def sortColors(colors, method):
    colors = (colors / 255) # colorsys uses 0 - 1 range 
    hsv_values = np.apply_along_axis(RGBToHSV, axis=1, arr=colors)

    if method == "HUE":
        hsv_values = hsv_values[hsv_values[:, 0].argsort()]
    elif method == "SATURATION":
        hsv_values = hsv_values[hsv_values[:, 1].argsort()]
    elif method == "BRIGHTNESS":
        hsv_values = hsv_values[hsv_values[:, 2].argsort()]
        
    rgb_values = np.apply_along_axis(HSVToRGB, axis=1, arr=hsv_values)
    rgb_values = (rgb_values * 255).astype(int) # convert color range back to 0 - 255
    return rgb_values
    
def RGBToHSV(color):
    return colorsys.rgb_to_hsv(color[0], color[1], color[2])
   
def HSVToRGB(color):
    return colorsys.hsv_to_rgb(color[0], color[1], color[2])

### Get frames for a certian range of frames and save it in scene

In [28]:
path = "frames256p/"
startFrame = 70796
endFrame = 77094 #196071
nframes = np.array(glob.glob(path+"*"))
scene = np.array([])

for e in nframes:
    if (int(e.split("_")[1].split(".")[0]) >= startFrame and
        int(e.split("_")[1].split(".")[0]) <= endFrame):
        scene = np.append(scene, e)
   
print(len(nframes), "frames found in total")
print(len(scene), "frames found in scene")

196071 frames found in total
6299 frames found in scene


### Parallel process all frames in array

In [8]:
ray.shutdown()
ray.init()

start_time = time.time()

@ray.remote
def f(x):
    image_path = scene[x]
    imagePIL = Image.open(image_path, "r")

    im, pixel_values = get_image_pixel(imagePIL, 1)

    sortedImage = sortColors(pixel_values, "HUE")
    
    return sortedImage

# Start range() tasks in parallel.
result_ids = []
for i in range(len(scene)):
    result_ids.append(f.remote(i))
    
# Wait for the tasks to complete and retrieve the results.
results = ray.get(result_ids)  # [0, 1, 2, 3]
print("--- took %s seconds ---" % (time.time() - start_time))

(f pid=6712) 
--- took 397.466933965683 seconds ---


In [9]:
len(results)

6298

### save array

In [19]:
nr = np.array(results)

In [20]:
np.save('sortedPixels.npy', nr)